In [1]:
import pandas as pd
# 1 - Importamos a biblioteca Pandas:
    # 1.1 - Damos um apelido para a biblioteca Pandas para pd.

dados = pd.read_csv('sementes.csv')
# 2 - Dados:
    # 2.1 - Salvamos em uma variavel chamada 'dados' o dataset 'sementes.csv'.
        # 2.1.1 - Utilizamos a função pd.read_csv() para ler os dados incluidos no arquivo .csv.

dados.head()
# 3 - Visualização dos Dados:
    # 3.1 - Utilizamos a função do pandas utilizando o dataframe 'dados' -> dados.head() para visualizar os dados do dataset 'sementes.csv'

,Área,Perímetro,Compacidade,Comprimento,Largura,Assimetria,Comprimento do sulco,Espécie
0,15.26,14.84,871.000,5.763,3.312,2.221,5.220,0
1,14.88,14.57,8.811,5.554,3.333,1.018,4.956,0
2,14.29,14.09,905.000,5.291,3.337,2.699,4.825,0
3,13.84,13.94,8.955,5.324,3.379,2.259,4.805,0
4,16.14,14.99,9.034,5.658,3.562,1.355,5.175,0


In [2]:
# Diferenciando X e Y:
 # X --> Informações de entradas.
 # y --> Alvo de identificação.

X = dados.drop(['Espécie'], axis = 1).values
# Instruções:

    #  1 - Dados é o dataframe que utilizamos.

    # 2 - Drop serve para cortar parte:
        # 2.1 - Quando axis = 1, ele manipula coluna;
        # 2.2 - Quando axis = 0, ele manipula linha.

    # 3 - Values serve para somente "pegar" os valores de 'Espécies'.

y = dados['Espécie'].values
# Instruções:

    # 1 - Utilizamos Dados como dataframe:
        # 1.1 - dados['Espécie'].values --> Coluna que coletaremos os valores.



In [3]:
from sklearn.model_selection import train_test_split
# 1 - Importamos somente a função train_test_split da biblioteca sklearn.model_selection

X_treino,X_teste,y_treino,y_teste = train_test_split(X,y,test_size=0.2)
# 2 - Dividimos os conjuntos de dados X e y em conjuntos de treinamento e teste, onde X_treino e Y_treino são os conjuntos de treinamento e X_teste e Y_teste são os conjuntos de teste.
# 2.1 - O parâmetro test_size=0.2 especifica que 20% dos dados serão reservados para o conjunto de teste.

In [4]:
import torch
# 1 - Importamos a biblioteca Torch

# 2 - Transformar Arrays em Tensores
    # - X_treino
    # - X_teste
    # - y_treino
    # - y_teste

X_treino = torch.FloatTensor(X_treino)

X_teste = torch.FloatTensor(X_teste)

y_treino = torch.LongTensor(y_treino)

y_teste = torch.LongTensor(y_teste)


In [5]:
import torch.nn as nn
# 1 - Importar os módulos da biblioteca de Redes Neurais do Torch 'nn'.
import torch.nn.functional as F
# 2 - Importar os módulos da biblioteca Fucional do Torch 'F' para funções de ativação.

In [6]:
class Modelo(nn.Module):
    
# 1.1 - Define uma classe chamada 'Modelo' que herda da classe 'nn.Module'.
# 1.2 - Esta classe serve como uma estrutura base para a definição de modelos de rede neural
# 1.3 - Utilizando a biblioteca PyTorch (nn.Module é uma classe base para todos os modelos de redes neurais em PyTorch).
    
    def __init__(self,entrada=7,camada_escondida1 = 14, camada_escondida2 = 49, saida = 3):
        # 2.1 - Define o método de inicialização (__init__) da classe Modelo.
        # 2.2 - Este método é chamado quando uma nova instância da classe Modelo é criada.
        # 2.3 - Ele define os parâmetros da rede neural, como o número de neurônios em cada camada.
        # 2.4 - Os parâmetros padrão especificam os valores padrão para o número de neurônios em cada camada caso nenhum valor seja explicitamente fornecido ao instanciar um objeto da classe Modelo.
        # 2.5 - Por exemplo, se nenhum valor for fornecido, a rede neural terá:
                # - 7 neurônios na camada de entrada ;
                # - 14 neurônios na primeira camada escondida ;
                # - 49 neurônios na segunda camada escondida ;
                # - 3 neurônios na camada de saída.
        super().__init__()
            # 3.1 - Chama o método __init__ da classe pai (nn.Module).
            # 3.2 - Isso é necessário para inicializar a classe pai e garantir que todos os recursos e métodos da classe pai estejam disponíveis para a classe Modelo.

        self.fc1 = nn.Linear(entrada, camada_escondida1)
            # 4.1 - Cria uma camada linear (fully connected) com 'entrada' neurônios na camada de entrada e 'camada_escondida1' neurônios na primeira camada escondida.
            # 4.2 - Esta camada é responsável por realizar a transformação linear dos dados de entrada.
        
        self.fc2 = nn.Linear(camada_escondida1, camada_escondida2)
            # 5.1 - Cria uma segunda camada linear (fully connected) com 'camada_escondida1' neurônios na entrada e 'camada_escondida2' neurônios na segunda camada escondida.
            # 5.2 - Esta camada é responsável por realizar a transformação linear dos dados da primeira camada escondida para a segunda.
        
        self.out = nn.Linear(camada_escondida2, saida)
            # 6.1 - Cria uma terceira camada linear (fully connected) com 'camada_escondida2' neurônios na entrada e 'saida' neurônios na camada de saída.
            # 6.2 - Esta camada é responsável por realizar a transformação linear dos dados da segunda camada escondida para a camada de saída.
    def forward(self, x):
        x = F.relu(self.fc1(x))

        x = F.relu(self.fc2(x))

        x = F.relu(self.out(x))

        return x

In [9]:
modelo_classificacao = Modelo()

funcao_objetivo = nn.CrossEntropyLoss()

otimizador = torch.optim.Adam(modelo_classificacao.parameters(), lr=0.01)

In [10]:
epocas = 100
custos = []
for i in range(epocas):
  y_predito = modelo_classificacao.forward(X_treino)
  custo = funcao_objetivo(y_predito, y_treino)
  custos.append(custo)

  otimizador.zero_grad()
  custo.backward()
  otimizador.step()

In [11]:
preds = []
with torch.no_grad():
    for val in X_teste:
        y_predito = modelo_classificacao.forward(val)
        preds.append(y_predito.argmax().item())
df = pd.DataFrame({'Y': y_teste, 'YHat': preds})
df['Correto'] = [1 if corr == pred else 0 for corr, pred in zip(df['Y'], df['YHat'])]
df

,Y,YHat,Correto
0,1,0,0
1,1,0,0
2,2,0,0
3,1,0,0
4,0,0,1
5,2,0,0
6,1,0,0
7,2,0,0
8,2,0,0
9,1,0,0
